In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import CountVectorizer

### Stop words

In [2]:
with open('stopwords-fr.json') as f:
    text_file = f.read()
    StopWord = json.loads(text_file)

### Dictionnaire de synonymes

In [3]:
with open("./thesaurus-v2.3/dic_synonymes.json") as f:
    data = f.read()
    dic_synonymes = json.loads(data)

### Get vocabulary and synonymes

In [4]:
def get_vocabulary_synonymes(text):
    #si le texte est vide, renvoie un dic vide
    if text == "":
        return {}
    #spliter le texte en phrases
    text = text.replace("\n", "").split(".")
    
    #compter les mots par occurence et enlever les stopwords
    vectorizer = CountVectorizer(stop_words=StopWord)
    X = vectorizer.fit_transform(text)

    frequence_mots = X.toarray()
    
    compte = np.sum(frequence_mots, axis=0)
    
    compte = list(compte)
    mots = vectorizer.get_feature_names()

    mots_occurence = pd.DataFrame(data={'compte' : compte, 'mots': mots})
    mots_occurence = mots_occurence.sort_values(by = 'compte', ascending=False)
    mots_occurence.iloc[:30]
    
    return {mot:dic_synonymes[mot] for mot in mots if mot in dic_synonymes}


In [ ]:
from flask import Flask, escape, request, jsonify

app = Flask(__name__, static_url_path='/static')

@app.route('/')
def root():
    return app.send_static_file('index_html.html')

@app.route('/vue')
def rootvue():
    response = app.send_static_file('index_vue.html')
    response.headers.add('Access-Control-Allow-Origin', '*')
    return response


@app.route('/api/syno', methods=['POST'])
def hello():
    text = request.get_json()
    mot_vers_synonymes = get_vocabulary_synonymes(text)
    return jsonify(mot_vers_synonymes)

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


127.0.0.1 - - [30/Jan/2020 10:26:15] "GET / HTTP/1.1" 200 -
